![LSC/Virgo Logo](http://www.ligo.org/images/logo02.gif)

# LIGO-Virgo EM Follow-Up Tutorial

by Leo P. Singer (NASA/GSFC) <leo.singer@ligo.org>

This document is [LIGO-G1500442-v6](https://dcc.ligo.org/G1500442).


## Abstract

This document explains how to receive, filter, and process gravitational-wave (GW) detection candidate alerts from Advanced LIGO and Virgo. We provide sample code in Python and document alternatives for users of other programming environments. You can download this document and run the code samples in [IPython Notebook](http://ipython.org/notebook.html).

## Introduction

The LIGO-Virgo data are analyzed in near real-time to search for GW transients due to compact binary coalescence (CBC) events or unmodeled "burst" sources. For each detection candidate, a series of alerts are produced and distributed via the [Gamma-ray Coordinates Network (GCN)](http://gcn.gsfc.nasa.gov).

![GCN network](http://gcn.gsfc.nasa.gov/GCN.gif)

The GCN system may already be familiar to some, as it is has been in use since the early 1990s to transmit times and coordinates of gamma-ray bursts (GRBs) detected by gamma-ray space missions to ground-based observers. However, there are three peculiariaties of GW observations that veteran GCN users should be aware of:

1. Unlike GRBs, GW sources cannot (always) be localized to a unique sky location; GW position reconstructions are (sometimes) multimodal and non-Gaussian. As a consequence, LIGO-Virgo GCN notices do *not* contain a RA, Dec, or error radius; instead they contain a pointer URL to a FITS file containing a probability sky map in the [HEALPix](http://healpix.sourceforge.net) all-sky projection.
1. GCN notices will produce LIGO-Virgo candidates that span a range of significances. The significance of a candidate is reported as the false alarm rate (FAR). The FAR of a detection candidate measures approximately how frequently an event of similar strenght occurs due to chance noise fluctuations or instrumental glitches. A FAR of 1/century ($\sim 3 \times 10^{-10}$ Hz) is generally regarded as sufficient for a "confident" detection. However, alerts will produced for (**FIXME**) all events with FAR≥1/month ($\sim 4\times10^{-7}$ Hz), such that there should be at least one alert per month on average.
1. The first few GW alerts will be distributed privately to LIGO MOU partners, over a purpose-built GCN connection. (**FIXME: link to description of policy**)

## 1. Sign up for GCN network

The first step is to sign up for the GCN network. You will need a computer with a static IP address and you will need to register the IP address and port number from which you will connect to the GCN network. (Note: You can have multiple GCN subscriptions on different IP addresses/ports).)

There is a [standard signup process for GCN](http://gcn.gsfc.nasa.gov/invitation.html), but only the standard notice types are supported. To receive the LIGO-Virgo GCNs, you must contact Scott Barthelmey (NASA/GSFC) directly. Fill in the example e-mail below with the name of your site, your contact information, and the IP address/port on which you want to receive GCNs.

    To: scott@lheamail.gsfc.nasa.gov
    Subject: LSC/Virgo GCN subscription

    Dear Scott,

    Please sign me up for receiving all LIGO-Virgo GCN notices
    (LVC_PRELIMINARY, LVC_INITIAL, and LVC_UPDATE). See below
    for site and connection details.

    Best,
    Edwin Hubble

    Site: Mount Wilson **FIXME**
    Contact: Edwin Hubble <hubble@obs.carnegiescience.edu> **FIXME**
    IP Address: 22.231.113.64:8096 **FIXME**
    Distribution Method: VOEvent/XML (all servers)

## 2. Sign up for a GraceDb robot password

You will need a *robot certificate* to download files that are linked from the GCN notices.

Follow the instructions at **FIXME** https://wiki.ligo.org/AuthProject/LIGOCARobotCertificate to request and obtain a robot certificate. Save the resulting `...key.pem` and `...cert.pem` file to the machine on which you have elected to receive GCNs.

**FIXME** Replace with instructions for populating .netrc file.

> New text will be...
>
> You will need a *robot password* to download files that are linked from the GCN notices. This is an automatically generated password that you can use to download files from GraceDb in a script. It is different from your log in for the GraceDb web site, but you use the GraceDb web site to manage your robot password. Here is how to obtain a robot password.
>
> 1. Got to https://gracedb.ligo.org/ and log in.
> 1. Click the **Options** tab (see screenshot below).
> ![GraceDb screenshot 1](https://dcc.ligo.org/G1500442/gracedb-screenshot-1.png)
> 1. Click the **Manage Password** link (see screenshot below).
> ![GraceDb screenshot 1](https://dcc.ligo.org/G1500442/gracedb-screenshot-2.png)
> 1. Click **Get me a password!** (See screenshot below.) ***Warning: if you had an old robot password, this will reset it and generate a new one.***
> ![GraceDb screenshot 1](https://dcc.ligo.org/G1500442/gracedb-screenshot-3.png)
> 1. Record the robot user name (which is the same as your user name for the GraceDb web site) and the generated robot password (see screenshot below).
> ![GraceDb screenshot 1](https://dcc.ligo.org/G1500442/gracedb-screenshot-4.png)
> 1. **Optional, but recommended:** For convenience, we suggset that you add the following line to your [~/.netrc](http://www.gnu.org/software/inetutils/manual/html_node/The-_002enetrc-File.html) file:
>
>   `machine gracedb.ligo.org login` *albert.einstein@LIGO.ORG* `password` *ABCDEabcde0123456789*
>
> replacing *albert.einstein@LIGO.ORG* with the robot user name and *ABCDEabcde0123456789* with the robot password. This will make it so that you can call [`curl`](http://linux.die.net/man/1/curl) or other HTTP downloader programs without explicitly providing the user name and password. To test your user name and password, you can run the following command:
>
>     curl --netrc https://gracedb.ligo.org/apibasic/

## 3. Install some dependencies

You will need to install a few third-party Python packages to run the example code in this tutorial. These include:

* [pygcn](https://pypi.python.org/pypi/pygcn) for connecting to GCN (alternatives: [comet](https://pypi.python.org/pypi/Comet))
* [requests](https://pypi.python.org/pypi/requests) for easy HTTP downloads in Python (many [alternatives](https://docs.python.org/2/library/urllib.html) in Python standard library)
* [healpy](https://pypi.python.org/pypi/healpy), for decoding HEALPix images (alternatives: [DS9](http://ds9.si.edu), [Aladin](http://aladin.u-strasbg.fr), [HEALPix bindings for C/C++/Fortran/Java/IDL](http://healpix.sourceforge.net), [reprojection with GammaPy](http://gammapy.readthedocs.org/en/v0.1/api/gammapy.image.healpix_to_image.html))
* [astropy](https://pypi.python.org/pypi/astropy) version 1.0.1 or newer (optional, for computing observability windows, etc.)

If you are on a Mac and use the [MacPorts](http://macports.org) package manager, you can install all of the above with the following command:

    $ sudo port install py27-gcn py27-lxml py27-requests py27-healpy py27-astropy

Otherwise, the fastest way to install the dependencies is with [`pip`](https://pip.pypa.io/en/latest/quickstart.html), a package manager that comes with most Python distributions. To install these packages with `pip`, run the following command:

    $ pip install pygcn lxml requests healpy astropy

## 4. Write GCN handler script

Now we'll write a GCN handler script. First, some imports...

In [ ]:
# Python standard library imports
import tempfile
import shutil
import sys
import glob

# Third-party imports
import gcn
import gcn.handlers
import gcn.notice_types
import requests
import healpy as hp

**FIXIME** ***use the robot password instead of the robot cert.*** Next, we need to record the path to the certificate from Step 2. We will assume it is in the current working directory, and glob for it.

In [ ]:
# Find the certificate and private key files
# (assumes that they are in the current working directory and
# have filenames ending in cert.pem and key.pem respectively).
certfile, = glob.glob('*cert.pem')
keyfile, = glob.glob('*key.pem')

Next, we'll write a function that we want to get called every time a GCN is received. We will use the `@gcn.handlers.include_notice_types` [https://docs.python.org/3/glossary.html#term-decorator](function decorator) to specify that we only want to process certain notice types. There are three notice types:

1. `LVC_PRELIMINARY`: Provides the time, significance, and basic parameters about a GW detection candidate. No localization information. Sent with a latency of a minute or so.
1. `LVC_INITIAL`: A rapid sky localization is available. Sent with a latency of a few minutes.
1. `LVC_UDPATE`: A refined sky localization is availaable. Sent with a latency of hours or more.

In the following example, we will process only the last two types (`LVC_INITIAL` and `LVC_UPDATE`), which contain links to sky map FITS files. The following handler function will parse out the URL of the FITS file, download it, and extract the probability sky map.

**FIXME** ***Untested, because the GCN VOEvents are currently malformed as XML documents (missing a closing tag).***

In [ ]:
def get_skymap(root):
    """
    Look up URL of sky map in VOEvent XML document,
    download sky map, and parse FITS file.
    """
    # Read out URL of sky map.
    # This will be something like
    # https://gracedb.ligo.org/api/events/M131141/files/bayestar.fits.gz
    skymap_url = root.find("./What/Param[@name='SKYMAP_URL_FITS_X509']").attrib['value']

    # Send HTTP request for sky map
    # FIXME: drop cert= stuff when robot user name and password go live
    # New code will be:
    # response = requests.get(skymap_url, stream=True)
    response = requests.get(skymap_url, stream=True, cert=(certfile, keyfile))

    # Uncomment to save VOEvent payload to file
    # open('example.xml', 'w').write(payload)

    # Raise an exception unless the download succeeded (HTTP 200 OK)
    response.raise_for_status()

    # Create a temporary file to store the downloaded FITS file
    with tempfile.NamedTemporaryFile() as tmpfile:
        # Save the FITS file to the temporary file
        shutil.copyfileobj(response.raw, tmpfile)
        tmpfile.flush()

        # Uncomment to save FITS payload to file
        # shutil.copyfileobj(reponse.raw, open('example.fits.gz', 'wb'))

        # Read HEALPix data from the temporary file
        skymap, header = hp.read_map(tmpfile.name, h=True, verbose=False)
        header = dict(header)

    # Done!
    return skymap, header


# Function to call every time a GCN is received.
# Run only for notices of type LVC_INITIAL or LVC_UPDATE.
@gcn.handlers.include_notice_types(
    gcn.notice_types.LVC_INITIAL,
    gcn.notice_types.LVC_UPDATE)
def process_gcn(payload, root):
    # Print the alert
    print('Got VOEvent:')
    print(payload)

    # Read out integer notice type (note: not doing anythin with this right now)
    notice_type = int(root.find("./What/Param[@name='Packet_Type']").attrib['value'])

    # Read sky map
    skymap, header = get_skymap(root)

Finally, we will listen for GCNs. You need to tell the `gcn.listen` function on what port you want to connect to the GCN network; this should be the same port that you provided to Scott Barthelmey in Section 1 above. You also need to tell `gcn.listen` which function to call whenever it receives an GCN; this will be the `process_gcn` function that we just defined.

When you run the following code snippet, the `gcn.listen` function will continue until you interrupt the program (by pressing the stop button in IPython Notebook, typing Control-C in the terminal, or sending a kill signal to your Python script).

Note: `gcn.listen` will automatically reconnect to the GCN network if the network connection is broken.

In [ ]:
# Listen for GCNs until the program is interrupted
# (killed or interrupted with control-C).
gcn.listen(port=8096, handler=process_gcn)

## Basic observability calculations with Astropy

That was fun! Now kill the listener: if you are running the example code in an IPython Notebook, press the stop button. If you copied the example code into a standalone Python script, then kill the script by typing control-C.

Now we are going to teach our GCN handler how to determine whether a gravitational-wave event is observable. We are goint to use the [astropy.coordinates](http://astropy.readthedocs.org/en/stable/coordinates/index.html) module of the [Astropy package](http://astropy.org) to do [observation planning in Python](http://astropy.readthedocs.org/en/stable/coordinates/observing-example.html). First, we will need to import a few extra Python modules:

In [ ]:
import numpy as np
import astropy.coordinates
import astropy.time
import astropy.units as u

Next, we are going to write a Python function that tells us how much of the probability is contained within the patch of sky that is immediately visible from a particular site on the Earth. The LIGO/Virgo probability sky maps are in the [HEALPix](http://healpix.sourceforge.net) projection. The probability maps take the form of a 1D array of values. Each entry in the array represents the probability contained within a quadrilateral pixel whose position on the sky is uniquely specified by the index in the array and the array's length. The [`pix2ang`](http://healpy.readthedocs.org/en/latest/generated/healpy.pixelfunc.pix2ang.html) function converts from pixel index to spherical polar coordinates; the function [`ang2pix`](http://healpy.readthedocs.org/en/latest/generated/healpy.pixelfunc.ang2pix.html) does the reverse.

![HEALPix projection](http://healpix.jpl.nasa.gov/images/healpixGridRefinement.jpg)

The LIGO/Virgo probability sky maps are always in equatorial coordinates. Once we have looked up the coordinates of the HEALPix pixels, we will use [the positional astronomy features of Astropy](http://astropy.readthedocs.org/en/stable/coordinates/observing-example.html) to transform those coordinates to an alt/az frame for a particular site on the Earth at a particular time. Then we can quickly determine which pixels are visible from that site at that time, and integrate (sum) the probability contained in those pixels.

Note: users may want to do something more sophisticated like determine how much of the probability is visible for at least a certain length of time. This example will illustrate one key function of HEALPix (looking up coordinates of the grid with `hp.pix2ang`) and some of the key positional astronomy functions with Astropy.

In [ ]:
def prob_observable(m, header):
    """
    Determine the integrated probability contained in a gravitational-wave
    sky map that is observable from a particular ground-based site at a
    particular time.

    Bonus: make a plot of probability versus UTC time!
    """

    # Determine resolution of sky map
    npix = len(m)
    nside = hp.npix2nside(npix)

    # Get time now
    time = astropy.time.Time.now()
    # Or at the time of the gravitational-wave event...
    # time = astropy.time.Time(header['MJD-OBS'], format='mjd')
    # Or at a particular time...
    # time = astropy.time.Time('2015-03-01 13:55:27')

    # Geodetic coordinates of observatory (example here: Mount Wilson)
    observatory = astropy.coordinates.EarthLocation(
        lat=34.2247*u.deg, lon=-118.0572*u.deg, height=1742*u.m)

    # Alt/az reference frame at observatory, now
    frame = astropy.coordinates.AltAz(obstime=time, location=observatory)

    # Look up (celestial) spherical polar coordinates of HEALPix grid.
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    # Convert to RA, Dec.
    radecs = astropy.coordinates.SkyCoord(
        ra=phi*u.rad, dec=(0.5*np.pi - theta)*u.rad)

    # Transform grid to alt/az coordinates at observatory, now
    altaz = radecs.transform_to(frame)

    # Where is the sun, now?
    sun_altaz = astropy.coordinates.get_sun(time).transform_to(altaz)

    # How likely is it that the (true, unknown) location of the source
    # is within the area that is visible, now? Demand that sun is at
    # least 18 degrees below the horizon and that the airmass
    # (secant of zenith angle approximation) is at least 2.5.
    prob = m[(sun_altaz.alt >= -18*u.deg) & (altaz.secz >= 2.5)].sum()

    # Done!
    return prob

Finally, we need to update our GCN handler to call this function.

In [ ]:
# Function to call every time a GCN is received.
# Run only for notices of type LVC_INITIAL or LVC_UPDATE.
@gcn.handlers.include_notice_types(
    gcn.notice_types.LVC_INITIAL,
    gcn.notice_types.LVC_UPDATE)
def process_gcn(payload, root):
    # Print the alert
    print('Got VOEvent:')
    print(payload)

    skymap, header = get_skymap(root)
    prob = prob_observable(skymap, header)
    print('Source has a %d%% chance of being observable now' % round(100 * prob))
    if prob > 0.5:
        pass # FIXME: perform some action

Let's run the new GCN handler now...

In [ ]:
# Listen for GCNs until the program is interrupted
# (killed or interrupted with control-C).
gcn.listen(port=8096, handler=process_gcn)

# Appendix: Full example code

Here is a complete, working GCN processing script. Copy it into a `.py` file and customize as needed.

In [ ]:
# Python standard library imports
import tempfile
import shutil
import sys
import glob


# Third-party imports
import gcn
import gcn.handlers
import gcn.notice_types
import requests
import healpy as hp
import numpy as np
import astropy.coordinates
import astropy.time
import astropy.units as u


# Find the certificate and private key files
# (assumes that they are in the current working directory and
# have filenames ending in cert.pem and key.pem respectively).
certfile, = glob.glob('*cert.pem')
keyfile, = glob.glob('*key.pem')


def get_skymap(root):
    """
    Look up URL of sky map in VOEvent XML document,
    download sky map, and parse FITS file.
    """

    # Read out URL of sky map.
    # This will be something like
    # https://gracedb.ligo.org/api/events/M131141/files/bayestar.fits.gz
    skymap_url = root.find("./What/Param[@name='SKYMAP_URL_FITS_X509']").attrib['value']

    # Send HTTP request for sky map
    # FIXME: drop cert= stuff when robot user name and password go live
    # New code will be:
    # response = requests.get(skymap_url, stream=True)
    response = requests.get(skymap_url, stream=True, cert=(certfile, keyfile))

    # Uncomment to save VOEvent payload to file
    # open('example.xml', 'w').write(payload)

    # Raise an exception unless the download succeeded (HTTP 200 OK)
    response.raise_for_status()

    # Create a temporary file to store the downloaded FITS file
    with tempfile.NamedTemporaryFile() as tmpfile:
        # Save the FITS file to the temporary file
        shutil.copyfileobj(response.raw, tmpfile)
        tmpfile.flush()

        # Uncomment to save FITS payload to file
        # shutil.copyfileobj(reponse.raw, open('example.fits.gz', 'wb'))

        # Read HEALPix data from the temporary file
        skymap, header = hp.read_map(tmpfile.name, h=True, verbose=False)
        header = dict(header)

    # Done!
    return skymap, header


def prob_observable(m, header):
    """
    Determine the integrated probability contained in a gravitational-wave
    sky map that is observable from a particular ground-based site at a
    particular time.

    Bonus: make a plot of probability versus UTC time!
    """

    # Determine resolution of sky map
    npix = len(m)
    nside = hp.npix2nside(npix)

    # Get time now
    time = astropy.time.Time.now()
    # Or at the time of the gravitational-wave event...
    # time = astropy.time.Time(header['MJD-OBS'], format='mjd')
    # Or at a particular time...
    # time = astropy.time.Time('2015-03-01 13:55:27')

    # Geodetic coordinates of observatory (example here: Mount Wilson)
    observatory = astropy.coordinates.EarthLocation(
        lat=34.2247*u.deg, lon=-118.0572*u.deg, height=1742*u.m)

    # Alt/az reference frame at observatory, now
    frame = astropy.coordinates.AltAz(obstime=time, location=observatory)

    # Look up (celestial) spherical polar coordinates of HEALPix grid.
    theta, phi = hp.pix2ang(nside, np.arange(npix))
    # Convert to RA, Dec.
    radecs = astropy.coordinates.SkyCoord(
        ra=phi*u.rad, dec=(0.5*np.pi - theta)*u.rad)

    # Transform grid to alt/az coordinates at observatory, now
    altaz = radecs.transform_to(frame)

    # Where is the sun, now?
    sun_altaz = astropy.coordinates.get_sun(time).transform_to(altaz)

    # How likely is it that the (true, unknown) location of the source
    # is within the area that is visible, now? Demand that sun is at
    # least 18 degrees below the horizon and that the airmass
    # (secant of zenith angle approximation) is at least 2.5.
    prob = m[(sun_altaz.alt >= -18*u.deg) & (altaz.secz >= 2.5)].sum()

    # Done!
    return prob


# Function to call every time a GCN is received.
# Run only for notices of type LVC_INITIAL or LVC_UPDATE.
@gcn.handlers.include_notice_types(
    gcn.notice_types.LVC_INITIAL,
    gcn.notice_types.LVC_UPDATE)
def process_gcn(payload, root):
    # Print the alert
    print('Got VOEvent:')
    print(payload)

    skymap, header = get_skymap(root)
    prob = prob_observable(skymap, header)
    print('Source has a %d%% chance of being observable now' % round(100 * prob))
    if prob > 0.5:
        pass # FIXME: perform some action here


# Listen for GCNs until the program is interrupted
# (killed or interrupted with control-C).
gcn.listen(port=8096, handler=process_gcn)